#### Concept of Givens rotation

The idea of Givens rotation is to `individually` turn subdiagonal entries into zeros using a series of orthogonal matrices and thereby doing QR decomposition

This procedure is illustrated using a matrix $A\in \mathbf{R}^{3 \times 3}$ where `x` indicates entries that are not necessarily zero

$$A:\begin{bmatrix} \times & \times & \times \\\times & \times & \times\\\times & \times & \times\end{bmatrix} \rightarrow G_1A\rightarrow \begin{bmatrix} \times & \times & \times \\ 0& \times & \times\\ \times & \times & \times\end{bmatrix}\rightarrow G_2G_1A\rightarrow\begin{bmatrix} \times & \times & \times \\ 0& \times & \times\\ 0 & \times & \times\end{bmatrix}\rightarrow G_3G_2G_1A\rightarrow \begin{bmatrix} \times & \times & \times \\ 0& \times & \times\\ 0 & 0 & \times\end{bmatrix}$$

For a dense matrix of size $n$, the method of Givens rotation requires up to $\frac{1}{2}n(n-1)$ transformations. However, when a matrix is sparse, the method can be more efficient than other QR methods

#### Structure of Givens rotation matrices

Each orthogonal matrix in Givens rotation would essentially be an identity matrix except for two rows and two columns

$$G=\begin{bmatrix}
1 & & & & & & & & & & \\
& \ddots & & & & & & & & & \\
& & 1 & & & & & & & & \\
& & & c & & & & s & & & \\
& & & & 1 & & & & & & \\
& & & & & \ddots & & & & & \\
& & & & & & 1 & & & & \\
& & & -s & & & & c & & & \\
& & & & & & & & 1 & & \\
& & & & & & & & & \ddots & \\
& & & & & & & & & & 1
\end{bmatrix}$$

If $c^2+s^2=1$, we can verify that $G$ is orthogonal

#### Sequential transformtion

For a matrix of interest $A\in \mathbf{R}^{n \times n}$, Givens rotation works column-wise and for each column the method starts from the highest non-zero subdiagonal entry

For example, it starts from the first column in matrix $A$

$$a_1=\begin{bmatrix}a_{11} \\ a_{21} \\ a_{31} \\ \vdots \\ a_{n1} \end{bmatrix}\rightarrow G_1a_1\rightarrow \begin{bmatrix}\alpha \\ 0 \\ a_{31} \\ \vdots \\ a_{n1} \end{bmatrix}$$

Since $G_1$ is orthogonal, the `norm` of $a_1$ before and after the transformation is preserved and we have $\alpha = \sqrt{a_{11}^2+a_{21}^2}$

For this case, we know the rotation matrix should have the following form

$$G_1=\begin{bmatrix}c & s & & & \\ -s & c & & & \\ & & 1 & & \\
& & & \ddots & \\ & & & & 1\end{bmatrix}$$

and we can work out that

$$c=\frac{a_{11}}{\alpha}, s=\frac{a_{21}}{\alpha}$$

This process repeats for the remaining subdiagonal entries of the first column, before moving on to the subsequent columns until all are taken care of

We can see that similar to Householder reflector, Givens rotation also does `full QR` by design (of the rotation matrix)

#### Applicability

Givens rotation works with tall, square and fat matrices, and full rank and rank deficient matrices. $R$ may not be in strict upper staircase form in case the matrix is rank deficient.

#### Example

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [ ]:
def givens_rotation(A):
    m, n = A.shape
    Q = np.identity(m)
    for col in range(min(m, n)):
        for row in range(col+1, m):
            if A[row, col] != 0:
                alpha = np.sqrt(A[col, col]**2 + A[row, col]**2)
                c = A[col, col] / alpha
                s = A[row, col] / alpha

                G = np.identity(m)
                G[row, col] = -s
                G[col, row] = s
                G[col, col] = c
                G[row, row] = c

                A = G @ A
                Q = Q @ G.T

    return Q, A

In [ ]:
mat_list = [
    'square_full', # 0
    'square_low_rank', # 1
    'tall_full', # 2
    'tall_low_rank', # 3
    'fat_full', # 4
    'fat_low_rank', # 5
    'ill-conditioned',  # 6
    'identity'] # 7

mat = mat_list[5]
epsilon = 1e-8

if mat == 'square_full':
    A = np.array([[1.0, 2.0, 3.0, 4.0],
                  [4.0, 1.0, 0.0, -1.0],
                  [3.0, 5.0, -2.0, 1.0],
                  [2.0, 0.0, 1.0, 2.0]])

elif mat == 'square_low_rank':
    A = np.array([[1.0, 2.0, 3.0, 3.0],
                  [4.0, 1.0, 0.0, 0.0],
                  [3.0, 5.0, -2.0, -2.0],
                  [2.0, 0.0, 1.0, 1.0]])

elif mat == 'tall_full':
    A = np.array([[1.0, 2.0, 3.0],
                  [4.0, 1.0, 0.0],
                  [3.0, 5.0, -2.0],
                  [2.0, 0.0, 1.0],
                  [5.1, 5.2, 5.3]])

elif mat == 'tall_low_rank':
    A = np.array([[1.0, 1.0, 3.0],
                  [4.0, 4.0, 0.0],
                  [3.0, 3.0, -2.0],
                  [2.0, 2.0, 1.0],
                  [5.1, 5.1, 5.3]])

elif mat == 'fat_full':
    A = np.array([[1.0, 1.2, 1.7, 1.1, 5.7],
                  [4.0, 4.3, 4.1, 2.2, 6.6],
                  [3.0, 3.4, 3.9, 3.3, 5.2]])

elif mat == 'fat_low_rank':
    A = np.array([[1.0, 1.0, 1.0, 1.1, 5.7],
                  [4.0, 4.0, 4.0, 2.2, 6.6],
                  [3.0, 3.0, 3.0, 3.3, 5.2]])

elif mat == 'ill-conditioned':
    A = np.array([[1, 1, 1],
                  [epsilon, 0, 0],
                  [0, epsilon, 0],
                  [0, 0, epsilon]])

elif mat == 'identity':
    A = np.identity(4)

Q, R = np.linalg.qr(A)
print("Q from NumPy:\n", Q)
print("\nR from NumPy:\n", R)

Q, R = givens_rotation(A)

print("\nOrthonormal basis Q:")
print(Q)

print("\nUpper triangular matrix R:")
print(R)

print(f"\nQ^TQ:\n{np.dot(Q.T, Q)}")
print(f"Norms: \n{np.linalg.norm(Q, axis=0)}")

print(f"\nOriginal A:\n{A}")

# Reconstruct A (full QR)
A_reconstructed = Q @ R
print(f"\nA_reconstructed from full QR:\n{A_reconstructed}")

# Reduced QR
n = A.shape[1]
A_reduced = np.dot(Q[:, :n], R[:n, :])
print("\nA_reconstructed from reduced QR:")
print(A_reduced)

print(f"\nReconstruction error from A_reconstructed:\n{np.linalg.norm(A - A_reconstructed)}")
print(f"\nReconstruction error from A_reduced:\n{np.linalg.norm(A - A_reduced)}")

Q from NumPy:
 [[-0.1961  0.9707  0.1387]
 [-0.7845 -0.0705 -0.6162]
 [-0.5883 -0.2296  0.7753]]

R from NumPy:
 [[-5.0990 -5.0990 -5.0990 -3.8831 -9.3547]
 [ 0.0000 -0.0000 -0.0000  0.1550  3.8740]
 [ 0.0000  0.0000  0.0000  1.3555  0.7555]]

Orthonormal basis Q:
[[ 0.1961 -0.1427  0.9701]
 [ 0.7845 -0.5708 -0.2425]
 [ 0.5883  0.8086  0.0000]]

Upper triangular matrix R:
[[ 5.0990  5.0990  5.0990  3.8831  9.3547]
 [ 0.0000  0.0000  0.0000  1.2557 -0.3758]
 [ 0.0000  0.0000  0.0000  0.5336  3.9291]]

Q^TQ:
[[ 1.0000  0.0000  0.0000]
 [ 0.0000  1.0000  0.0000]
 [ 0.0000  0.0000  1.0000]]
Norms: 
[ 1.0000  1.0000  1.0000]

Original A:
[[ 1.0000  1.0000  1.0000  1.1000  5.7000]
 [ 4.0000  4.0000  4.0000  2.2000  6.6000]
 [ 3.0000  3.0000  3.0000  3.3000  5.2000]]

A_reconstructed from full QR:
[[ 1.0000  1.0000  1.0000  1.1000  5.7000]
 [ 4.0000  4.0000  4.0000  2.2000  6.6000]
 [ 3.0000  3.0000  3.0000  3.3000  5.2000]]

A_reconstructed from reduced QR:
[[ 1.0000  1.0000  1.0000  1.1000 